<a href="https://colab.research.google.com/github/sruthi2498/PanaromaAnalysis/blob/main/PanoramaCreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import os
import shutil

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
video_path = "background.avi"
selected_frames_folder = "selected_frames"

In [ ]:
if os.path.isdir(selected_frames_folder):
    shutil.rmtree(selected_frames_folder);
os.mkdir(selected_frames_folder);

In [ ]:
cap = cv.VideoCapture(video_path)
counter = 0;
orb = cv2.ORB_create();
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False);
j = 1
_, last = cap.read();
last = rescale(last);
cv2.imwrite(selected_frames_folder + str(counter).zfill(5) + ".png", last);

kp1, des1 = orb.detectAndCompute(last, None);

cutoff = 20; 
selected_frames = [j]

prev = None;
while True:
    j+=1
    ret, frame = cap.read();
    if not ret:
        break;
    frame = rescale(frame);
    kp2, des2 = orb.detectAndCompute(frame, None);
    matches = bf.knnMatch(des1, des2, k=2);
    # lowe's ratio
    good = []
    for m,n in matches:
        if m.distance < 0.75*n.distance:
            good.append(m);
    if len(good) < cutoff:
        # swap and save
        counter += 1;
        last = frame;
        kp1 = kp2;
        des1 = des2;
        cv2.imwrite(selected_frames_folder + str(counter).zfill(5) + ".png", last);
        print("New Frame: " + str(counter));
        selected_frames.append(j)
    prev = frame;

counter += 1;
cv2.imwrite(selected_frames_folder + str(counter).zfill(5) + ".png", prev);

# check number of saved frames
print("Counter: " + str(counter));

In [ ]:
final_panorama_path = "panorama.png"

In [ ]:
stitcher = cv.Stitcher.create()
ret, pano = stitcher.stitch(frames)
if ret==cv.STITCHER_OK:
  cv2_imshow(pano)
  cv2.imwrite(final_panorama_path, pano);